# Quick start with Ablator

Welcome to the Ablator tutorial! In this chapter, you will learn how to run an experiment from scratch. We will provide a simple demo to see what it looks like to run Ablator. You are also welcome to download this demo @[Colab](https://colab.research.google.com/drive/127l02PicoLxAZ3b_JL9eVpMxQ_UQKgiG?usp=sharing) or [Github](https://github.com/SeanXiaoby/ablator-fork/tree/tutorial-demos/examples/demo-basics-usage-vscode)

Let's get started!

## Installing

We assume you have installed Python (Python >= 3.10) and pip on your local machine. Please use the following command to install Ablator:

```bash
pip install ablator
```

## Preparations

Below is a summary of the steps to launch an experiment with Ablator:

- Set up experiment configurations
- Define your idea - a model and datasets
- Wrap model with model wrapper and launch experiment

To showcase that Ablator takes care of training and evaluation of your idea (so you don't need to write training and evaluation scripts), in this demo, we will define a simple LeNet-5 model and classic MNIST dataset, wrap them with Ablator, and run the experiment.

Before we start, let's import the necessary packages:

```python
import shutil
import argparse
from typing import Any, Callable, Dict
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score
```

### Set up configurations

Configuring an experiment requires defining several configuration objects: model configuration, training configuration, optimizer configuration, and run configuration.

In this chapter, we set up the experiment configurations using traditional Python objects and classes. The following code shows how to set up configurations for Ablator:

```python
from ablator import ModelConfig, TrainConfig, OptimizerConfig, RunConfig,
                     configclass, Literal, ModelWrapper, ProtoTrainer

@configclass
class SimpleConfig(ModelConfig):
    name: Literal["simplenet"]

@configclass
class SimpleRunConfig(RunConfig):
    model_config: SimpleConfig

run_config = SimpleRunConfig(
    experiment_dir = "/tmp/dir",
    train_config = TrainConfig(
        dataset = "mnist",
        batch_size = 64,
        epochs = 10,
        scheduler_config = None,
        rand_weights_init = False,
        optimizer_config = OptimizerConfig(
            name = "sgd",
            arguments = {
                "lr": 0.001,
                "momentum": 0.1
            }
        )
    ),
    model_config = SimpleConfig(name = "simplenet"),
    metrics_n_batches = 200,
    device= "cpu",
    amp=False
)
```

Note that there are multiple ways to set up the configuration. To learn more, go to the [Configuration Basics](./Configuration-Basics.ipynb) tutorial.

### Define your idea - a model and datasets

This is where you define your novel idea (e.g. an original model or dataset). The following code shows how to define a model and datasets (note that you can customize this step to your needs):

- Model detail:
  - Define a simple CNN module using components from PyTorch packages.
  - We will include the simple CNN as a part of the main model, define the loss function, and define the forward pass.

```python
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(120, 84)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = self.relu3(self.fc1(x))
        x = self.relu4(self.fc2(x))
        x = self.fc3(x)
        return x


class MyModel(nn.Module):
    def __init__(self, config: SimpleConfig) -> None:
        super().__init__()
        self.model = SimpleCNN()
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, labels, custom_input=None):
        # custom_input is for demo purposes only, defined in the dataset wrapper
        out = self.model(x)
        loss = self.loss(out, labels)
        if labels is not None:
            loss = self.loss(out, labels)

        out = out.argmax(dim=-1)
        out = out.reshape(-1,1)
        labels = labels.reshape(-1,1)
        
        return {"y_pred": out, "y_true": labels}, loss
```

- Dataset detail: we create the training & validation data loaders from the MNIST dataset. Data preprocessing includes normalization and transformations to tensor:

```python
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.MNIST(root='./datasets', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./datasets', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)
```

- An evaluation function is definded here for Ablator to evaluate the model.

```python
def my_accuracy(y_true, y_pred):
    return accuracy_score(y_true.flatten(), y_pred.flatten())

```

### Wrap model with model wrapper and launch experiment

As a final step, use the model wrapper to wrap the main model (this will add boiler-plate codes on the training and evaluation of your model on your dataset; we also add accuracy as an evaluation metric to the evaluation step) and launch Ablator.

```python
class MyModelWrapper(ModelWrapper):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def make_dataloader_train(self, run_config: SimpleRunConfig):
        return trainloader

    def make_dataloader_val(self, run_config: SimpleRunConfig):
        return testloader

    def evaluation_functions(self) -> Dict[str, Callable]:
        return {"accuracy_score": my_accuracy}

if __name__ == "__main__":
    wrapper = MyModelWrapper(model_class=MyModel)
    shutil.rmtree(run_config.experiment_dir, ignore_errors=True)    # Remove previous experiment results if existed to avoid experiment existed error when launching the experiment
    ablator = ProtoTrainer(
        wrapper=wrapper,
        run_config=run_config,
    )
    ablator.launch()
```

For Jupyter notebooks, directly run the above codes in the notebook. You can so save the above code snippets in a Python script and run it with the following command:

```shell
python <your_script_name>.py
```

If Ablator is successfully launched, you should see information printed on the console!

## Access the results

The training process (experiment results: training, evaluation results) is recorded by Ablator and is saved in the experiment directory (`run_config.experiment_dir`). You can access the training results of the model by using the following codes:

```shell
cd /tmp/dir/
cat results.json
```

You should see the training results from each epoch.

You can also visualize the results by using Tensorboard in Jupyter Notebook:

```python
# Load the TensorBoard extension
import tensorboard
%load_ext tensorboard

# Start TensorBoard
%tensorboard --logdir /tmp/dir/dashboard/tensorboard
```

## Next steps

Ablator is far beyond what we show you in this tutorial. Please refer to the following chapters for more features and functionalies of Ablator!